In [1]:
import pandas as pd
import numpy as np

In [65]:
cabinet_raw = pd.read_csv("signal_data/cabinet_8_16.csv")
signal_raw = pd.read_csv("signal_data/signals_8_16.csv")

In [3]:
cabinet_raw.head(5)

,CABINET_ID,COA_CABINET_ADDRESS_ID,signal,CABINET_CORNER,UPS,COA_ADDRESS_ID,CABINET_LONGITUDE,CABINET_TYPE,CABINET_LATITUDE,CREATED_BY,...,CONTROLLER_IP,IP_COMM_STATUS,COMM_STATUS_DATETIME_UTC,HUB_ID,COMM_TYPE,SWITCH_IP,SWITCH_SEQUENCE#,RING_ID,CIRCUIT_NAME,ART_BOX
0,1,NaN,332: LAVACA ST / 3RD ST,NE,No,0.0,-97.746387,332,30.265837,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,2,NaN,742: COLORADO ST / 3RD ST,SW,No,0.0,-97.745254,332,30.265727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,3,NaN,959: COLORADO ST / 4TH ST,SW,No,0.0,-97.744945,332,30.266473,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
3,4,NaN,288: LAVACA ST / 4TH ST,NW,No,0.0,-97.745999,332,30.267003,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
4,5,NaN,326: COLORADO ST / 2ND ST,NE,No,0.0,-97.745412,332,30.264758,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [7]:
cabinet_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 34 columns):
CABINET_ID                  935 non-null int64
COA_CABINET_ADDRESS_ID      0 non-null float64
signal                      935 non-null object
CABINET_CORNER              932 non-null object
UPS                         935 non-null object
COA_ADDRESS_ID              923 non-null float64
CABINET_LONGITUDE           914 non-null float64
CABINET_TYPE                935 non-null object
CABINET_LATITUDE            914 non-null float64
CREATED_BY                  0 non-null float64
MODIFIED_BY                 55 non-null object
CREATED_DATE                935 non-null object
MODIFIED_DATE               935 non-null object
SIGNAL_ID                   935 non-null int64
LOCATION                    935 non-null object
LOCATION: Street 1          933 non-null object
LOCATION: Street 2          0 non-null float64
LOCATION: City              931 non-null object
LOCATION: State             935 no

In [6]:
signal_raw.head(5)

,SIGNAL_ID,SIGNAL_TYPE,OWNER,TURN_ON_DATE,MODIFIED_DATE,COMMENT,CONTROL,ATD_LOCATION_ID,LOCATION_NAME,TURN_ON_YEAR,...,COUNT_WORK_ORDERS,SUM_LOOP_DETECTORS,SUM_VIDEO_DETECTORS,SUM_GRIDSMART_DETECTORS,SUM_NO_DETECTORS,DETECTION_TYPE,COMM_TYPE,task_order,ip_swith_cell_temp,comm_type_temp
0,2,TRAFFIC,COA,03/18/1965,07/09/2018 8:10am,NaN,PRIMARY,LOC16-000005,GUADALUPE ST / LAMAR BLVD,-5,...,2,3.0,NaN,0.0,0.0,LOOP,Switch,NaN,NaN,NaN
1,3,TRAFFIC,COA,10/27/1976,09/21/2016 12:00am,NaN,PRIMARY,LOC16-000010,LAMAR BLVD / 51ST ST,7,...,9,0.0,NaN,0.0,0.0,LOOP,Switch,NaN,NaN,NaN
2,4,TRAFFIC,COA,08/02/1951,09/21/2016 12:00am,NaN,PRIMARY,LOC16-000015,LAMAR BLVD / NORTH LOOP BLVD,-18,...,8,4.0,NaN,0.0,0.0,LOOP,Switch,NaN,NaN,NaN
3,5,TRAFFIC,COA,NaN,09/21/2016 12:00am,NaN,PRIMARY,LOC16-000020,KOENIG LN / LAMAR BLVD,0,...,8,6.0,NaN,0.0,0.0,LOOP,Switch,NaN,NaN,NaN
4,6,TRAFFIC,COA,11/18/1970,09/21/2016 12:00am,NaN,PRIMARY,LOC16-000025,LAMAR BLVD / DENSON DR,1,...,1,2.0,NaN,0.0,0.0,LOOP,Switch,NaN,NaN,NaN


In [8]:
signal_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1117 entries, 0 to 1116
Data columns (total 57 columns):
SIGNAL_ID                        1117 non-null int64
SIGNAL_TYPE                      1117 non-null object
OWNER                            1111 non-null object
TURN_ON_DATE                     897 non-null object
MODIFIED_DATE                    1117 non-null object
COMMENT                          62 non-null object
CONTROL                          1117 non-null object
ATD_LOCATION_ID                  1117 non-null object
LOCATION_NAME                    1117 non-null object
TURN_ON_YEAR                     1117 non-null int64
SIGNAL_STATUS                    1117 non-null object
SWITCH_COMM                      1039 non-null object
CONTROLLER_IP                    923 non-null object
IP_COMM_STATUS                   1117 non-null object
MODIFIED_BY                      491 non-null object
CREATED_DATE                     69 non-null object
asset_id_generator               1117 

In [66]:
# pick useful columns from two datasets
signal = signal_raw[["SIGNAL_ID", "PRIMARY_SIGNAL_ID", "CONTROL"]]
cabinet = cabinet_raw[["SIGNAL_ID", "CABINET_ID"]]

In [56]:
# add cabinet ID to signals dataframe
# signal["CABINET_ID"] = ""

In [67]:
# filter out secondary signals
secondary = signal.loc[signal["CONTROL"] == "SECONDARY"].copy()

In [68]:
# filter out primary signals
primary = signal.loc[signal["CONTROL"] == "PRIMARY"].copy()

In [87]:
secondary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 6 to 1025
Data columns (total 3 columns):
SIGNAL_ID            135 non-null int64
PRIMARY_SIGNAL_ID    132 non-null float64
CONTROL              135 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 3.7+ KB


In [85]:
primary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 982 entries, 0 to 1116
Data columns (total 3 columns):
SIGNAL_ID            982 non-null int64
PRIMARY_SIGNAL_ID    0 non-null float64
CONTROL              982 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 26.9+ KB


### 1. Fill cabinet IDs for secondary signals. 

In [91]:
# match cabinet ID to secondary signals using the primary 
# signal IDs of secondary signals
secondary_cab = secondary.merge(cabinet, 
                                left_on = "PRIMARY_SIGNAL_ID", 
                                right_on = "SIGNAL_ID", 
                                how = "left").copy()

In [100]:
secondary_cab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 0 to 135
Data columns (total 5 columns):
SIGNAL_ID_x          136 non-null int64
PRIMARY_SIGNAL_ID    133 non-null float64
CONTROL              136 non-null object
SIGNAL_ID_y          126 non-null float64
CABINET_ID           126 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.8+ KB


In [99]:
secondary_cab["SIGNAL_ID_x"].value_counts()

672     2
1038    1
584     1
345     1
809     1
597     1
340     1
339     1
594     1
336     1
603     1
591     1
334     1
77      1
588     1
331     1
858     1
97      1
582     1
104     1
915     1
618     1
621     1
878     1
468     1
627     1
884     1
546     1
380     1
904     1
       ..
548     1
652     1
654     1
147     1
916     1
917     1
407     1
983     1
667     1
412     1
157     1
416     1
931     1
623     1
167     1
427     1
173     1
686     1
687     1
944     1
946     1
691     1
693     1
695     1
683     1
697     1
699     1
448     1
449     1
768     1
Name: SIGNAL_ID_x, Length: 135, dtype: int64

In [102]:
secondary_cab.loc[secondary_cab["SIGNAL_ID_x"] == 672]

,SIGNAL_ID_x,PRIMARY_SIGNAL_ID,CONTROL,SIGNAL_ID_y,CABINET_ID
66,672,671.0,SECONDARY,671.0,696.0
67,672,671.0,SECONDARY,671.0,697.0


In [72]:
secondary_cab["CABINET_ID"].value_counts().head(5)

79.0     3
776.0    3
741.0    3
767.0    3
701.0    3
Name: CABINET_ID, dtype: int64

In [73]:
# slice out trailing 0
secondary_cab["CABINET_ID"] = secondary_cab["CABINET_ID"].astype(str)
secondary_cab["CABINET_ID"] = secondary_cab["CABINET_ID"].str.split('.', expand = True)[0]

In [74]:
secondary_cab["CABINET_ID"].value_counts().head(5)

nan    10
701     3
79      3
741     3
767     3
Name: CABINET_ID, dtype: int64

### 2. map cabinet IDs to primary signals


In [75]:
primary_cab = primary.merge(cabinet, 
                                left_on = "SIGNAL_ID", 
                                right_on = "SIGNAL_ID", 
                                how = "left").copy()

In [76]:
primary_cab.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 996 entries, 0 to 995
Data columns (total 4 columns):
SIGNAL_ID            996 non-null int64
PRIMARY_SIGNAL_ID    0 non-null float64
CONTROL              996 non-null object
CABINET_ID           921 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 35.0+ KB


In [77]:
primary_cab["CABINET_ID"].value_counts().head(5)

727.0    1
163.0    1
394.0    1
227.0    1
694.0    1
Name: CABINET_ID, dtype: int64

In [78]:
primary_cab["CABINET_ID"] = primary_cab["CABINET_ID"].astype(str)
primary_cab["CABINET_ID"] = primary_cab["CABINET_ID"].str.split('.', expand = True)[0]

In [108]:
type(primary_cab[""])

SyntaxError: invalid syntax (<ipython-input-108-b4ae0c1cdc7b>, line 1)

In [79]:
primary_cab["CABINET_ID"].value_counts().head(5)

nan    75
623     1
556     1
571     1
708     1
Name: CABINET_ID, dtype: int64

In [107]:
primary_cab["SIGNAL_ID"].value_counts()

671     2
479     2
25      2
513     2
653     2
785     2
828     2
245     2
871     2
90      2
94      2
914     2
486     2
462     2
354     1
352     1
355     1
356     1
357     1
358     1
351     1
359     1
353     1
4085    1
350     1
349     1
348     1
347     1
346     1
344     1
       ..
735     1
736     1
752     1
766     1
764     1
763     1
761     1
760     1
759     1
758     1
757     1
756     1
755     1
754     1
753     1
751     1
737     1
750     1
749     1
748     1
747     1
745     1
744     1
743     1
742     1
741     1
740     1
739     1
738     1
2       1
Name: SIGNAL_ID, Length: 982, dtype: int64

### 3. put primary and secondary signals (with cab ID) together

In [80]:
signal_cab = primary_cab.append(secondary_cab, ignore_index=True, sort=False)

In [81]:
signal_cab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132 entries, 0 to 1131
Data columns (total 6 columns):
SIGNAL_ID            996 non-null float64
PRIMARY_SIGNAL_ID    133 non-null float64
CONTROL              1132 non-null object
CABINET_ID           1132 non-null object
SIGNAL_ID_x          136 non-null float64
SIGNAL_ID_y          126 non-null float64
dtypes: float64(4), object(2)
memory usage: 44.3+ KB
